In [1]:
import os

In [2]:
%pwd

'/Users/ravina/Desktop/CustomerChurnPrediction/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/ravina/Desktop/CustomerChurnPrediction'

In [5]:
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)  #this is not python class but dataclass, here you can define the veriables without using self keyword.
class DataTransformationConfig:
    root_dir:Path
    data_path: Path

In [6]:
from CustomerChurnPrediction.constants import *
from CustomerChurnPrediction.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        

    def get_data_transformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,

        )

        return data_transformation_config

In [8]:
import os
from CustomerChurnPrediction import logger
from sklearn.model_selection import train_test_split
import pandas as pd 
from sklearn.preprocessing import StandardScaler,LabelEncoder
import imblearn
from imblearn.over_sampling import SMOTE





In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_splitting(self):
        df = pd.read_csv(self.config.data_path)
        
        # Separate the target variable 'Exited'
        y = df['Exited']
        X=df.drop(['Exited'], axis=1)
        
        numr = ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary']
        catg =['Geography', 'Gender', 'NumOfProducts', 'IsActiveMember', 'sufficient_balance', 'is_CreditScore_low']
        
        numerical_transformer = StandardScaler()
        categoricals= []

        # Apply LabelEncoder to each categorical column
        for column in catg:
            le = LabelEncoder()
            X[column] = le.fit_transform(X[column])
            categoricals.append((column, le))

        
        X[numr] = numerical_transformer.fit_transform(X[numr])   # Apply StandardScaler to numerical columns
        transformed_df = pd.concat([X[numr], X[catg]], axis=1)   # Concatenate numerical and encoded categorical columns
        
        # Split into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(transformed_df, y, test_size=0.3, stratify=y,random_state=42)
        
        
        smote = SMOTE(sampling_strategy='minority')    # Apply SMOTE to training set
        X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

        train_df = pd.concat([X_train_balanced, y_train_balanced], axis=1)   # Concatenate the encoded categorical, scaled numerical and target y.
        test_df = pd.concat([X_test, y_test], axis=1)   # Concatenate the test set with target y.

        # Save train and test sets to CSV
        train_df.to_csv(os.path.join(self.config.root_dir, 'train.csv'), index=False)
        test_df.to_csv(os.path.join(self.config.root_dir, 'test.csv'), index=False)

        logger.info("Data has been split into training and test sets.")
        logger.info(f"Training set shape: {train_df.shape}")
        logger.info(f"Test set shape: {test_df.shape}")


In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    raise e


[2024-04-30 14:26:18,765:INFO:yaml file:config/config.yaml loaded successfully]
[2024-04-30 14:26:18,766:INFO:yaml file:params.yaml loaded successfully]
[2024-04-30 14:26:18,768:INFO:yaml file:Schema.yaml loaded successfully]
[2024-04-30 14:26:18,768:INFO:created directory at:artifacts]
[2024-04-30 14:26:18,769:INFO:created directory at:artifacts/data_transformation]
[2024-04-30 14:26:18,867:INFO:Data has been split into training and test sets.]
[2024-04-30 14:26:18,867:INFO:Training set shape: (11146, 12)]
[2024-04-30 14:26:18,868:INFO:Test set shape: (3000, 12)]
